# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load cities.csv and create dataframe
file_path = '../weatherpy/output_data/cities.csv'
cities_df = pd.read_csv(file_path)
cities_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,2980080,Saint-François,96,FR,1597839556,33,46.42,3.91,87.80,9.17
1,6201424,Mataura,90,NZ,1597839556,95,-46.19,168.86,45.00,1.01
2,3466704,Castro,100,BR,1597839432,96,-24.79,-50.01,53.85,5.28
3,3445026,Vila Velha,20,BR,1597839408,78,-20.33,-40.29,78.01,10.29
4,5848280,Kapaa,40,US,1597839557,78,22.08,-159.32,78.80,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store latitude and longitude in locations
locations = cities_df[['Lat', 'Lon']]

In [4]:
# Plot map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = cities_df['Humidity'], dissipating = False,
                                max_intensity = cities_df['Humidity'].max(), point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create new datagrame to narrow cities_df to ideal conditions
ideal_cities_df = cities_df.loc[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80) & 
                                (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0)]

# Remove rows that dont meet all ideal conditions
ideal_cities_df.dropna()
ideal_cities_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
9,546105,Nikolskoye,0,RU,1597839558,43,59.70,30.79,71.60,6.71
150,3984997,San Quintín,0,MX,1597839591,78,30.48,-115.95,73.58,4.00
189,3445459,Várzea da Palma,0,BR,1597839601,49,-17.60,-44.73,77.92,7.99
311,1529651,Altay,0,CN,1597839627,25,47.87,88.12,73.31,9.44
353,4021858,Guerrero Negro,0,MX,1597839634,73,27.98,-114.06,74.79,4.83
405,3695844,Lagunas,0,PE,1597839644,86,-5.23,-75.68,74.75,1.39
479,5509851,Pahrump,0,US,1597839657,35,36.21,-115.98,79.00,1.16
485,3353383,Rundu,0,NaN,1597839658,17,-17.93,19.77,75.18,7.31
493,1516048,Khovd,0,MN,1597839659,28,48.01,91.64,74.37,2.95
529,1786657,Yinchuan,0,CN,1597839666,30,38.47,106.27,78.62,9.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create new dataframe hotel_df using ideal_cities_df
hotel_df = ideal_cities_df.loc[:, ['City ID', 'City', 'Cloudiness', 'Country', 'Humidity', 'Lat', 'Lon', 
                                   'Max Temp', 'Wind Speed']]

# Add new column to hotel_df
hotel_df['Hotel Name'] = ''

# Setup base url
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Create parameters dictionary for hotels within 5000 meters
params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

In [7]:
# Use lat/lon to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat and lon from the dataframe
    lat = row['Lat']
    lon = row['Lon']
    
    # Change location each iteration while leaving original params in place
    params['location'] = f'{lat}, {lon}'
    
    # make request and convert to json
    response = requests.get(base_url, params = params).json()
    #print(json.dumps(response, indent = 4, sort_keys = True))
    
    # Error handling
    results = response['results']
    
    try:
        print(f"{results[0]['name']} is the closest to {results[0]['vicinity']}")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print(f'Missing Field/Result...Skipping')
    
    print(f'-----------------------------------------------------------------------------------------------')

print(f'-----------------------------------------End of Search-----------------------------------------')



Missing Field/Result...Skipping
-----------------------------------------------------------------------------------------------
Old Mill Hotel is the closest to Ensenada
-----------------------------------------------------------------------------------------------
Hotel Casa Grande e Restaurante Forno e Fogão is the closest to R. Dr. Ensch, 865 - CENTRO, Várzea da Palma
-----------------------------------------------------------------------------------------------
Wenzhou Hotel is the closest to Altay, Tuanjie Rd, 10号区242栋
-----------------------------------------------------------------------------------------------
Hotel Los Caracoles is the closest to Calzada, De La República s/n, Estado 30, Guerrero Negro
-----------------------------------------------------------------------------------------------
Hospedaje Eco is the closest to Avenida Padre Lucero 1404 Lagunas, San Antonio de la Laguna
--------------------------------------------------------------------------------------------

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))